<a href="https://colab.research.google.com/github/rohansb10/pyspark/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
battings = pd.read_csv(r"https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_batting_card.csv")
bowings = pd.read_csv(r"https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_bowling_card.csv")
details = pd.read_csv(r"https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_details.csv")
summary = pd.read_csv(r"https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_summary.csv")


In [ ]:
# show the all of columns in pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
battings.head(3)

,season,match_id,match_name,home_team,away_team,venue,city,country,current_innings,innings_id,name,fullName,runs,ballsFaced,minutes,fours,sixes,strikeRate,captain,isNotOut,runningScore,runningOver,shortText,commentary,link
0,2024.0,1422119,CSK v RCB,CSK,RCB,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai,India,RCB,1,V Kohli,Virat Kohli,21,20,46,0,1,105.00,False,False,"{'wickets': 4, 'runs': 77}",11.2,c Ravindra b Mustafizur Rahman,tiki-taka catch between Rahane and Ravindra at...,NaN
1,2024.0,1422119,CSK v RCB,CSK,RCB,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai,India,RCB,1,F du Plessis,Faf du Plessis,35,23,21,8,0,152.17,True,False,"{'wickets': 1, 'runs': 41}",4.3,c Ravindra b Mustafizur Rahman,The Fizz strikes in his first over this IPL. T...,NaN
2,2024.0,1422119,CSK v RCB,CSK,RCB,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai,India,RCB,1,RM Patidar,Rajat Patidar,0,3,2,0,0,0.00,False,False,"{'wickets': 2, 'runs': 41}",4.6,c &dagger;Dhoni b Mustafizur Rahman,double strike for the Fizz. He has been done i...,NaN


In [28]:
!pip install pyspark

In [29]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, current_date, datediff, when
from pyspark.sql.types import StringType, IntegerType, DateType

# Step 1: Create a Spark session
spark = SparkSession.builder.appName("ETL Process Example").getOrCreate()

# -----------------------------------
# Extract: Creating Sample Data
# -----------------------------------

# Manually creating some sample data
data = [
    ("John", "1990-05-15", 60000),
    ("Alice", "1985-10-30", 80000),
    ("Bob", "2000-07-20", 45000),
    ("Emma", "1995-09-10", 70000)
]

# Defining schema for the DataFrame
schema = ["name", "birthdate", "salary"]

# Creating a DataFrame from the sample data
df = spark.createDataFrame(data, schema)

# Casting birthdate column to DateType
df = df.withColumn("birthdate", col("birthdate").cast(DateType()))

# -----------------------------------
# Transform: Processing the Data
# -----------------------------------

# Step 1: Calculate Age from Birthdate
df = df.withColumn("age", (datediff(current_date(), col("birthdate")) / 365.25).cast("int"))

# Step 2: Categorize Salary into ranges
# Using 'when' for conditional column creation
df = df.withColumn(
    "salary_category",
    when(col("salary") < 50000, "Low")
    .when((col("salary") >= 50000) & (col("salary") < 70000), "Medium")
    .otherwise("High")
)

# -----------------------------------
# Load: Save the Transformed Data
# -----------------------------------

# Show the transformed data (to confirm it's correct)
df.show()

# Write the transformed data to a CSV file
df.write.csv("transformed_output.csv", header=True)

# Stop the Spark session
spark.stop()


+-----+----------+------+---+---------------+
| name| birthdate|salary|age|salary_category|
+-----+----------+------+---+---------------+
| John|1990-05-15| 60000| 34|         Medium|
|Alice|1985-10-30| 80000| 38|           High|
|  Bob|2000-07-20| 45000| 24|            Low|
| Emma|1995-09-10| 70000| 29|           High|
+-----+----------+------+---+---------------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CreateTableSchemaExample") \
    .getOrCreate()

# Define the schema
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", FloatType(), True)
])

# Create a DataFrame with the defined schema
data = [
    ("Alice", 30, 50000.0),
    ("Bob", 25, 60000.0),
    ("Cathy", 35, 70000.0),
    ("David", 28,None )
]

df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.show()

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("employees")

# Run SQL queries on the temporary view
print("Run SQL queries on the temporary view")
result = spark.sql("SELECT * FROM employees WHERE age > 30")
result.show()

# printSchema
df.printSchema()

# Stop the SparkSession
spark.stop()


+-----+---+-------+
| name|age| salary|
+-----+---+-------+
|Alice| 30|50000.0|
|  Bob| 25|60000.0|
|Cathy| 35|70000.0|
|David| 28|   NULL|
+-----+---+-------+

Run SQL queries on the temporary view
+-----+---+-------+
| name|age| salary|
+-----+---+-------+
|Cathy| 35|70000.0|
+-----+---+-------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = true)



In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

# Step 1: Initialize Spark session
spark = SparkSession.builder.appName("PandasToPySpark").getOrCreate()

# Step 2: Sample Pandas DataFrame (you can use your existing Pandas DataFrame)
pandas_df = pd.read_csv("https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_batting_card.csv")

# Step 3: Convert Pandas DataFrame to PySpark DataFrame
spark_df = spark.createDataFrame(pandas_df)

# Step 4: Show the PySpark DataFrame
spark_df.show(5)


+------+--------+----------+---------+---------+--------------------+-------+-------+---------------+----------+------------+--------------+----+----------+-------+-----+-----+----------+-------+--------+--------------------+-----------+--------------------+--------------------+----+
|season|match_id|match_name|home_team|away_team|               venue|   city|country|current_innings|innings_id|        name|      fullName|runs|ballsFaced|minutes|fours|sixes|strikeRate|captain|isNotOut|        runningScore|runningOver|           shortText|          commentary|link|
+------+--------+----------+---------+---------+--------------------+-------+-------+---------------+----------+------------+--------------+----+----------+-------+-----+-----+----------+-------+--------+--------------------+-----------+--------------------+--------------------+----+
|2024.0| 1422119| CSK v RCB|      CSK|      RCB|MA Chidambaram St...|Chennai|  India|            RCB|         1|     V Kohli|   Virat Kohli|  21|

pyspark read

In [ ]:
import urllib.request
from pyspark.sql import SparkSession

# Step 1: Download the CSV files locally
url_batting = "https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_batting_card.csv"
url_bowling = "https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_bowling_card.csv"
url_details = "https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_details.csv"
url_summary = "https://raw.githubusercontent.com/rohansb10/pyspark/refs/heads/main/ipl%2024%20data/season_summary.csv"

urllib.request.urlretrieve(url_batting, "season_batting_card.csv")
urllib.request.urlretrieve(url_bowling, "season_bowling_card.csv")
urllib.request.urlretrieve(url_details, "season_details.csv")
urllib.request.urlretrieve(url_summary, "season_summary.csv")

# Step 2: Initialize a Spark session
spark = SparkSession.builder.appName("IPL_Data_Analysis").getOrCreate()

# Step 3: Read the local CSV files into PySpark DataFrames
battings = spark.read.csv("season_batting_card.csv", header=True, inferSchema=True)
bowlings = spark.read.csv("season_bowling_card.csv", header=True, inferSchema=True)
details = spark.read.csv("season_details.csv", header=True, inferSchema=True)
summary = spark.read.csv("season_summary.csv", header=True, inferSchema=True)



In [ ]:
# Show the schema of each dataset
print("Battings Schema:")
battings.printSchema()

print("Bowlings Schema:")
bowlings.printSchema()

print("Details Schema:")
details.printSchema()

print("Summary Schema:")
summary.printSchema()


Battings Schema:
root
 |-- season: double (nullable = true)
 |-- match_id: integer (nullable = true)
 |-- match_name: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- current_innings: string (nullable = true)
 |-- innings_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- fullName: string (nullable = true)
 |-- runs: integer (nullable = true)
 |-- ballsFaced: integer (nullable = true)
 |-- minutes: string (nullable = true)
 |-- fours: integer (nullable = true)
 |-- sixes: integer (nullable = true)
 |-- strikeRate: string (nullable = true)
 |-- captain: boolean (nullable = true)
 |-- isNotOut: boolean (nullable = true)
 |-- runningScore: string (nullable = true)
 |-- runningOver: double (nullable = true)
 |-- shortText: string (nullable = true)
 |-- commentary: string (nullable = true)
 |-- link

In [ ]:
# Show the first 5 rows of each dataset
print("Battings Data:")
battings.show(5)

print("Bowlings Data:")
bowlings.show(5)

print("Details Data:")
details.show(5)

print("Summary Data:")
summary.show(5)

Battings Data:
+------+--------+----------+---------+---------+--------------------+-------+-------+---------------+----------+------------+--------------+----+----------+-------+-----+-----+----------+-------+--------+--------------------+-----------+--------------------+--------------------+----+
|season|match_id|match_name|home_team|away_team|               venue|   city|country|current_innings|innings_id|        name|      fullName|runs|ballsFaced|minutes|fours|sixes|strikeRate|captain|isNotOut|        runningScore|runningOver|           shortText|          commentary|link|
+------+--------+----------+---------+---------+--------------------+-------+-------+---------------+----------+------------+--------------+----+----------+-------+-----+-----+----------+-------+--------+--------------------+-----------+--------------------+--------------------+----+
|2024.0| 1422119| CSK v RCB|      CSK|      RCB|MA Chidambaram St...|Chennai|  India|            RCB|         1|     V Kohli|   Vi

In [ ]:
# Count the number of rows in each dataset
print(f"Battings has {battings.count()} rows.")
print(f"Bowlings has {bowlings.count()} rows.")
print(f"Details has {details.count()} rows.")
print(f"Summary has {summary.count()} rows.")

# Describe statistics of numerical columns (like runs, wickets, etc.)
print("Battings Summary Stats:")
battings.describe().show()

print("Bowlings Summary Stats:")
bowlings.describe().show()


Battings has 928 rows.
Bowlings has 709 rows.
Details has 14357 rows.
Summary has 74 rows.
Battings Summary Stats:
+-------+------+------------------+----------+---------+---------+--------------------+-------------+-------+---------------+------------------+-------------+-------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+
|summary|season|          match_id|match_name|home_team|away_team|               venue|         city|country|current_innings|        innings_id|         name|     fullName|              runs|        ballsFaced|           minutes|             fours|             sixes|        strikeRate|        runningScore|       runningOver|           shortText|          commentary|                link|
+-------+------+------------------+----------+---------+---------+--------------------+-------------+----